In [92]:
# Block 1:
from __future__ import annotations
import json
import yfinance as yf
import types
from agno.agent import Agent
from agno.db.sqlite import SqliteDb
from agno.models.google import Gemini
from agno.tools import tool
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.memory import MemoryTools
from agno.workflow import Step, Workflow, StepInput, StepOutput, Parallel, Loop
from dotenv import load_dotenv
from json import loads
from typing import Dict, Any, Optional, Tuple


Load environment variables from a .env file

The following variables must be defined:
- GOOGLE_API_KEY
- NEWSAPI_KEY
- FRED_API_KEY

In [93]:
# Block 2:
load_dotenv()

DEFAULT_MODEL_ID = "gemini-2.5-flash"
DEFAULT_MODEL = Gemini(id=DEFAULT_MODEL_ID)

DEFAULT_AGENT_KWARGS = {
    "model": DEFAULT_MODEL,
    "exponential_backoff": True,
    "retries": 5,
    "delay_between_retries": 5,  # Initial delay in seconds
}

## Define Agents

### Earnings Agent

In [94]:
# Block 3:
earnings_agent = Agent(
    name="Earnings Agent",
    instructions=[
        "You are an earnings analysis agent for financial analysis.",
        "The user will provide a company name or stock symbol. Retrieve and summarize key insights from financial filings and earnings reports (such as SEC EDGAR) for the provided company or symbol.",
        "Focus on analyzing:",
        "1. Revenue and earnings growth trends",
        "2. Key financial ratios (P/E, debt-to-equity, ROE, etc.)",
        "3. Cash flow analysis and liquidity position",
        "4. Segment performance and business unit analysis",
        "5. Management guidance and forward-looking statements",
        "6. Risk factors and regulatory compliance issues",
        "7. Recent acquisitions, divestitures, or strategic initiatives",
        "Present your findings in a structured format with clear insights and implications for investment decisions.",
        "Highlight both positive and negative trends with supporting data from the filings.",
    ],
    db=SqliteDb(db_file="earnings_agent.db"),
    add_datetime_to_context=True,
    markdown=True,
    **DEFAULT_AGENT_KWARGS
)

### Market Data Agent

In [95]:
# Block 4:
@tool(name="fetch_quote", description="Get latest stock price and metadata for a symbol.")
def fetch_quote(symbol: str) -> str:
    """Fetch the latest quote for a stock symbol."""
    try:
        t = yf.Ticker(symbol)
        info = getattr(t, "fast_info", {}) or {}
        # fast_info is dict-like in recent yfinance versions
        price = info.get("last_price")

        if price is None:
            hist = t.history(period="5d", interval="1d", auto_adjust=True)
            if not hist.empty:
                price = float(hist["Close"].iloc[-1])

        result = {
            "symbol": symbol.upper(),
            "last_price": None if price is None else float(price),
            "currency": info.get("currency"),
            "exchange": info.get("exchange"),
        }
        return json.dumps(result)
    except Exception as e:
        return json.dumps({"symbol": symbol, "error": str(e)})


@tool(name="fetch_ohlcv", description="Get last 5 OHLCV rows for a symbol.")
def fetch_ohlcv(symbol: str, period: str = "1y", interval: str = "1d") -> str:
    """Fetch OHLCV history for a stock symbol."""
    try:
        df = yf.download(symbol, period=period, interval=interval, progress=False, auto_adjust=True, threads=False)
        if df.empty:
            return json.dumps({"symbol": symbol.upper(), "rows": 0, "tail5": []})
        tail = df.tail(5).reset_index()
        tail["Date"] = tail["Date"].astype(str)
        result = {
            "symbol": symbol.upper(),
            "rows": int(len(df)),
            "period": period,
            "interval": interval,
            "tail5": [
                {
                    "Date": r["Date"],
                    "Open": float(r["Open"]),
                    "High": float(r["High"]),
                    "Low": float(r["Low"]),
                    "Close": float(r["Close"]),
                    "Adj Close": float(r.get("Adj Close", r["Close"])),
                    "Volume": int(r["Volume"]),
                }
                for _, r in tail.iterrows()
            ],
        }
        return json.dumps(result)
    except Exception as e:
        return json.dumps({"symbol": symbol, "error": str(e)})

TOOLS = [fetch_quote, fetch_ohlcv]

market_data_agent = Agent(
    name="Market Data Agent",
    instructions=[
        "You are a financial market data agent.",
        "When the user provides a stock symbol or company name, call fetch_quote or fetch_ohlcv "
        "to fetch and summarize market data.",
        "Be clear about symbol, time range, and assumptions.",
    ],
    tools=TOOLS,
    db=SqliteDb(db_file="market_data_agent.db"),
    add_datetime_to_context=True,
    markdown=True,
    **DEFAULT_AGENT_KWARGS
)

### News Agent

In [96]:
# Block 5:
news_agent = Agent(
    name="News Agent",
    instructions=[
        "You are a financial analysis assistant."
        "The user will provide the name of a company or its stock ticker symbol. Please identify the most recent and relevant news articles for financial analysis.",
        "First search for recent news articles about the company or stock ticker symbol provided by the user.",
        "Then analyze the news articles and classify tem as positive, negative, or neutral in terms of their potential impact on the company's stock price.",
        "Create a list item for each news article, including these columns: 'Title', 'Source', 'URL', 'Date', 'Sentiment', and 'Summary'.",
        """Example:
        - Title: Example News Article
            - Source: Example News Source
            - URL: https://example.com/news/article
            - Date: 2023-01-01
            - Sentiment: Positive
            - Summary: This is an example summary of the news article.
        - Title: Another News Article
            - Source: Another News Source
            - URL: https://example.com/news/another-article
            - Date: 2023-01-02
            - Sentiment: Negative
            - Summary: This is another example summary of a different news article.
        """,
        "Respond only the list of news articles in markdown format.",
    ],
    db=SqliteDb(db_file="news_agent.db"),
    tools=[DuckDuckGoTools(fixed_max_results=5)],
    tool_call_limit=5,
    add_history_to_context=True,
    add_datetime_to_context=True,
    markdown=True,
    **DEFAULT_AGENT_KWARGS
)

### Memory Agent

In [97]:
# Block 6:
memory_instructions = """Capture any memory that can be used for financial analysis. Examples include:
- news articles
- stock performance data
- market trends
- analyst opinions
Tag or categorize the information with relevant keywords for easy retrieval later.
If multiple pieces of information are provided, store them as separate memories.
Do not give your opinion or analysis. Just store or retrieve the information as requested.,

You have access to Think, Add Memory, Update Memory, Delete Memory, and Analyze tools that will help you financial analysis data called memories and analyze their operations.
Use these tools as frequently as needed to successfully complete memory management tasks.

## How to use the Think, Memory Operations, and Analyze tools:             
          
1. **Think**
- Purpose: A scratchpad for planning memory operations, brainstorming memory content, and refining your approach. You never reveal your     
"Think" content to the user.           
- Usage: Call `think` whenever you need to figure out what memory operations to perform, analyze requirements, or decide on strategy.       
          
2. **Get Memories**                    
- Purpose: Retrieves a list of memories from the database.                         
- Usage: Call `get_memories` when you need to retrieve memories.                   
          
3. **Add Memory**                      
- Purpose: Creates new memories in the database with specified content and metadata.                    
- Usage: Call `add_memory` with memory content and optional topics when you need to store new information.                                  
          
4. **Update Memory**                   
- Purpose: Modifies existing memories in the database by memory ID.        
- Usage: Call `update_memory` with a memory ID and the fields you want to change. Only specify the fields that need updating.               
          
5. **Delete Memory**                   
- Purpose: Removes memories from the database by memory ID.                
- Usage: Call `delete_memory` with a memory ID when a memory is no longer needed or requested to be removed.                                
          
6. **Analyze**                         
- Purpose: Evaluate whether the memory operations results are correct and sufficient. If not, go back to "Think" or use memory operations   
with refined parameters.               
- Usage: Call `analyze` after performing memory operations to verify:      
    - Success: Did the operation complete successfully?                    
    - Accuracy: Is the memory content correct and well-formed?             
    - Completeness: Are all required fields populated appropriately?       
    - Errors: Were there any failures or unexpected behaviors?             
          
**Important Guidelines**:              
- Do not include your internal chain-of-thought in direct user responses.  
- Use "Think" to reason internally. These notes are never exposed to the user.                          
- When you provide a final answer to the user, be clear, concise, and based on the memory operation results.                                
- If memory operations fail or produce unexpected results, acknowledge limitations and explain what went wrong.                             
- Always verify memory IDs exist before attempting updates or deletions.   
- Use descriptive topics and clear memory content to make memories easily searchable and understandable.
You can refer to the examples below as guidance for how to use each tool.  
          
### Examples
          
#### Example 1: Adding multiple news articles as memories

User: <multiple articles and metadata about stock performance and market trends>
Think:  I should store each article and its metadata. I should create a memory with this information and use relevant topics for easy       
retrieval.
Add Memory: memory="<article1 information>, <article1 metadata>, <article1 sentiment>", topics=["<company name>", "<stock symbol>", "news", "stock performance", "market trends", "<date>", "<sentiment>"]
Add Memory: memory="<article2 information>, <article2 metadata>, <article2 sentiment>", topics=["<company name>", "<stock symbol>", "news", "stock performance", "market trends", "<date>", "<sentiment>"]
Add Memory: memory="<article3 information>, <article3 metadata>, <article3 sentiment>", topics=["<company name>", "<stock symbol>", "news", "stock performance", "market trends", "<date>", "<sentiment>"]
Analyze: Successfully created a memory with article information for each article provided. The topics are well-chosen for future retrieval. This should help with future
news-related requests.

Final Answer: Noted. I've stored the articles information. I'll remember the key details for future reference.

#### Example 2: Adding 1 news article as memories

User: <article information and metadata about stock performance and market trends>
Think:  I should store the article and its metadata. I should create a memory with this information and use relevant topics for easy       
retrieval.
Add Memory: memory="<article information>, <article metadata>, <article sentiment>", topics=["<company name>", "<stock symbol>", "news", "stock performance", "market trends", "<date>", "<sentiment>"]
Analyze: Successfully created a memory with article information. The topics are well-chosen for future retrieval. This should help with future
news-related requests.

Final Answer: Noted. I've stored the article information. I'll remember the key details for future reference.

#### Example 3: Updating Existing Information

User: Whoops, that was a mistake. The sentiment for the Business Insider's article written about MSFT in 2025-10-05 should be "neutral" instead of "positive".
Think: I need to find the specific memory related to this article and update its sentiment.
Update Memory: memory_id="matching_memory_id", memory="Business Insider's article about MSFT in 2025-10-05",          
topics=["MSFT", "Business Insider", "2025-10-05", "neutral"]
Analyze: Successfully updated the sentiment for the Business Insider's article about MSFT. The content now accurately reflects the corrected sentiment.

Final Answer: I've updated the sentiment for the Business Insider's article about MSFT to "neutral". Let me know if there's anything else you'd like to adjust.
          
#### Example 4: Removing Outdated Information                              

User: Please forget all Microsoft data older than 2023.
Think: The user wants me to delete old Microsoft data since it's no longer relevant. I should find and remove those memories.
Delete Memory: memory_id="microsoft_data_memory_id1"
Delete Memory: memory_id="microsoft_data_memory_id2"
Analyze: Successfully deleted the outdated Microsoft data memory. The old information won't interfere with future requests.

Final Answer: I've removed your old Microsoft data. Feel free to share any new information when you're ready, and I'll store the updated information.

#### Example 5: Retrieving Memories    
          
User: Latest MSFT news?
Think: The user wants to retrieve financial analysis data about Microsoft. I should use the get_memories tool to retrieve the memories.
Get Memories:
Analyze: Successfully retrieved the memories about Microsoft. The memories are relevant to the user's request criteria.

Final Answer: I've retrieved the memories about Microsoft. The latest news includes information about its stock performance, market trends, and other relevant financial data.
"""

memory_agent = Agent(
    name="Memory Agent",
    session_id="financial_memory_session",  # Use a fixed session ID to maintain memory across runs
    user_id="financial_user",  # Use a fixed user ID for consistent user identity
    instructions=[
        "You are a memory agent that stores and retrieves information for financial analysis.",
        "User may provide new information related to financial analysis. Store this information in a structured format.",
        "Always acknowledge if any information is stored.",
        "User may request information about a specific company or stock ticker symbol. Upon such requests, retrieve and summarize all relevant stored information related to that company or stock ticker symbol.",
    ],
    tools=[
        MemoryTools(
            db=SqliteDb(db_file="memory_agent_memories.db"),
            instructions=memory_instructions,
        )
    ],
    add_datetime_to_context=True,
    markdown=True,
    **DEFAULT_AGENT_KWARGS
)

### Investment Research Agent

In [98]:
# Block 7:
investment_research_agent = Agent(
    name="Investment Research Agent",
    instructions=[
        "You are an investment research agent for financial analysis.",
        "The user will provide a stock symbol or company name. Plan a research workflow for the provided stock symbol, including which data to gather, which agents to consult, and how to synthesize the findings.",
        "Create a comprehensive research plan that includes:",
        "1. Market data analysis (price trends, volume, technical indicators)",
        "2. News sentiment analysis (recent news articles and their impact)",
        "3. Earnings and financial filings review (SEC filings, quarterly reports)",
        "4. Risk assessment and market positioning",
        "5. Investment recommendation framework",
        "Structure your response as a detailed research workflow with specific steps and data sources.",
        "Provide actionable insights and recommendations based on your analysis framework.",
    ],
    db=SqliteDb(db_file="investment_research_agent.db"),
    add_datetime_to_context=True,
    markdown=True,
    **DEFAULT_AGENT_KWARGS
)

### Preprocessing Agent

In [99]:
# Block 8:
preprocessing_agent = Agent(
    name="Preprocessing agent",
    instructions=[
        "You are a financial analysis assistant."
        "Your task is to preprocess prompt inputs for downstream analysis.",
        "You respond in strict JSON format.",
        "Given raw text input, perform the following preprocessing steps:",
        "- Identify the ticker symbol for the company mentioned in the text.",
        "- Extract the intent of the user's request without losing context. Convert it into a concise action item.",
        "- Determine the type of data needed for analysis (The only valid values are: ['earnings', 'news', 'market']).",
        "Examples:",
        """Input: "Can you provide the latest news on Microsoft?"
Output: {"ticker": "MSFT", "action_item": "Provide the latest news on Microsoft", "data_types": ["news"]}""",
        """Input: "I need the recent earnings report for Apple."
Output: {"ticker": "AAPL", "action_item": "Retrieve the recent earnings report for Apple", "data_types": ["earnings"]}""",
        """Input: "What's the current market status of Tesla?"
Output: {"ticker": "TSLA", "action_item": "Get the current market status of Tesla", "data_types": ["market"]}""",
        """Input: "Perform a comprehensive analysis of Amazon.",
Output: {"ticker": "AMZN", "action_item": "Perform a comprehensive analysis of Amazon", "data_types": ["earnings", "news", "market"]}""",
    ],
    use_json_mode=True,
    **DEFAULT_AGENT_KWARGS
)

### Evaluator Agent

In [100]:
# Block 9:
# Evaluator Agent Implementation
# This agent:
# 1. Evaluates analysis quality (completeness, accuracy, clarity)
# 2. Provides constructive feedback for improvement
# 3. Works with Investment Research Agent for iterative refinement
# 4. Implements the Evaluator-Optimizer workflow pattern

evaluator_agent = Agent(
    name="Evaluator Agent",
    instructions=[
        "You are an evaluation agent for financial research outputs.",
        "Evaluate drafts for completeness, accuracy, and clarity.",
        "Provide actionable feedback with specific, constructive suggestions.",
        "If data is missing or uncertain, recommend concrete next steps to resolve gaps.",
        "Always return a JSON object with this schema:\n"
        "{"
        '  "scores": {"completeness": float, "accuracy": float, "clarity": float, "overall": float},'
        '  "feedback": {"strengths": [str], "gaps": [str], "suggestions": [str]},'
        '  "actions": {"priority_fixes": [str], "checks": [str], "followups": [str]},'
        '  "ready_for_delivery": bool'
        "}",
        "Scores must be 0.0-1.0. The 'overall' is the mean of the three dimensions.",
        "Mark 'ready_for_delivery' true only if overall ≥ 0.85 and no critical gaps remain.",
        "Be objective, concise, and specific. Generic advice is ok.",
    ],
    db=SqliteDb(db_file="evaluator_agent.db"),
    add_datetime_to_context=True,
    markdown=True,
    **DEFAULT_AGENT_KWARGS,
)


def _safe_json_loads(s: str) -> Dict[str, Any]:
    """Best-effort JSON parser with graceful fallback."""
    try:
        return json.loads(s)
    except Exception:
        return {"raw_response": s}


def _compute_overall(scores: Dict[str, float]) -> float:
    """Compute overall as mean of completeness, accuracy, clarity if not provided."""
    keys = ["completeness", "accuracy", "clarity"]
    vals = [float(scores.get(k, 0.0)) for k in keys]
    return sum(vals) / max(len(vals), 1)


def _threshold_met(feedback: Dict[str, Any], min_overall: float = 0.85) -> bool:
    """Decide if optimization is needed."""
    try:
        scores = feedback.get("scores", {})
        overall = float(scores.get("overall", _compute_overall(scores)))
        return overall >= min_overall and not feedback.get("feedback", {}).get("gaps")
    except Exception:
        return False


def build_eval_prompt(draft: str, context: Optional[str] = None) -> str:
    parts = [
        "Evaluate the following financial analysis draft.",
        "Return ONLY the JSON object as per the schema.",
        "",
        "== DRAFT ==",
        draft.strip(),
    ]
    if context and context.strip():
        parts.extend(["", "== CONTEXT ==", context.strip()])
    return "\n".join(parts)


def build_optimizer_prompt(draft: str, eval_json: Dict[str, Any]) -> str:
    return (
        "You are preparing an optimization plan for the Investment Research Agent.\n"
        "Produce a structured revision plan (JSON) with:\n"
        "{"
        ' "objective": str,'
        ' "edits": [{"type": "add|revise|remove", "target": str, "rationale": str, "detail": str}],'
        ' "data_requests": [str],'
        ' "acceptance_criteria": [str]'
        "}\n\n"
        "== CURRENT DRAFT ==\n"
        f"{draft.strip()}\n\n"
        "== EVALUATION ==\n"
        f"{json.dumps(eval_json, indent=2)}\n\n"
        "Return ONLY the JSON object."
    )


def evaluate_analysis(draft: str, context: Optional[str] = None) -> Dict[str, Any]:
    """
    Run the evaluator on a draft. Returns parsed JSON feedback (or raw fallback).
    """
    prompt = build_eval_prompt(draft, context)
    resp = evaluator_agent.run(prompt)
    data = _safe_json_loads(getattr(resp, "content", str(resp)))
    if "scores" in data and "overall" not in data["scores"]:
        data["scores"]["overall"] = _compute_overall(data["scores"])
    return data


def build_revision_plan(draft: str, eval_feedback: Dict[str, Any]) -> Dict[str, Any]:
    """
    Produce a JSON revision plan (no rewriting yet). Keeps Evaluator separate from generator.
    """
    prompt = build_optimizer_prompt(draft, eval_feedback)
    resp = evaluator_agent.run(prompt)
    return _safe_json_loads(getattr(resp, "content", str(resp)))


def evaluator_optimizer_cycle(
    draft: str,
    context: Optional[str] = None,
    symbol_or_name: Optional[str] = None,
    max_rounds: int = 2,
    min_overall: float = 0.85,
) -> Tuple[str, Dict[str, Any]]:
    """
    Minimal Evaluator-Optimizer loop:
      1) Evaluate draft
      2) If threshold not met, build plan & optimize with Investment Research Agent
      3) Re-evaluate the new draft
    Returns (final_draft, last_feedback_json)
    """
    current = draft
    last_feedback = {}

    for _ in range(max_rounds):
        last_feedback = evaluate_analysis(current, context)
        if _threshold_met(last_feedback, min_overall=min_overall):
            return current, last_feedback

        plan = build_revision_plan(current, last_feedback)
        # optimize_with_investment_agent(current, plan, symbol_or_name)
        # TO-DO: Call agent to optimize with
        improved = "not available"
        # If optimizer not available or returns empty, stop gracefully
        if not improved or "not available" in improved.lower():
            return current, last_feedback
        current = improved

    # Final evaluation after last round
    last_feedback = evaluate_analysis(current, context)
    return current, last_feedback

## Workflows

### Multi-Agent Router

In [101]:
# Block 10:
# Agent registry for easy routing
AGENT_REGISTRY = {
    "investment": investment_research_agent,
    "earnings": earnings_agent,
    "news": news_agent,
    "memory": memory_agent,
    "market": market_data_agent,
    "evaluator": evaluator_agent,
}


def route_content(content_type: str, data: str, **kwargs) -> Dict[str, Any]:
    """
    Implement Routing workflow pattern:
    Direct content to the right specialist (earnings, news, or market analyzers)
    
    Args:
        content_type: Type of content (earnings, news, market, memory, investment, evaluator)
        data: Content data to analyze
        **kwargs: Additional parameters to pass to the agent
        
    Returns:
        Dict containing:
        - agent_name: Name of the agent that processed the request
        - content_type: Type of content routed
        - response: Analysis from the appropriate specialist agent
        - status: Success or error status
        
    Example:
        >>> result = route_content("market", "AAPL")
        >>> print(result["response"])
    """
    # Normalize content type
    content_type_lower = content_type.lower().strip()
    
    # Map content types to agents
    routing_map = {
        "earnings": "earnings",
        "financial": "earnings",
        "filings": "earnings",
        "sec": "earnings",
        "quarterly": "earnings",
        "annual": "earnings",
        
        "news": "news",
        "sentiment": "news",
        "articles": "news",
        "headlines": "news",
        
        "market": "market",
        "price": "market",
        "stock": "market",
        "quote": "market",
        "ohlcv": "market",
        "technical": "market",
        
        "memory": "memory",
        "remember": "memory",
        "recall": "memory",
        "store": "memory",
        
        "investment": "investment",
        "research": "investment",
        "analysis": "investment",
        "recommendation": "investment",
        "general": "investment",
        
        "evaluator": "evaluator",
        "evaluate": "evaluator",
        "quality": "evaluator",
        "assess": "evaluator",
    }
    
    # Find the appropriate agent
    agent_key = routing_map.get(content_type_lower)
    
    if not agent_key:
        return {
            "agent_name": "Unknown",
            "content_type": content_type,
            "response": f"Error: Unknown content type '{content_type}'. "
                       f"Valid types: {', '.join(set(routing_map.values()))}",
            "status": "error"
        }
    
    # Get the agent from registry
    agent = AGENT_REGISTRY.get(agent_key)
    
    if not agent:
        return {
            "agent_name": agent_key,
            "content_type": content_type,
            "response": f"Error: Agent '{agent_key}' not found in registry",
            "status": "error"
        }
    
    try:
        # Route to the appropriate agent
        response = agent.run(data, **kwargs)
        
        # Extract content from response (handle different response formats)
        if hasattr(response, 'content'):
            response_content = response.content
        elif isinstance(response, dict) and 'content' in response:
            response_content = response['content']
        else:
            response_content = str(response)
        
        return {
            "agent_name": agent.name,
            "content_type": content_type,
            "response": response_content,
            "status": "success"
        }
        
    except Exception as e:
        return {
            "agent_name": agent.name if agent else "Unknown",
            "content_type": content_type,
            "response": f"Error during routing: {str(e)}",
            "status": "error"
        }


def intelligent_routing(content: str, context: Optional[str] = None) -> Dict[str, Any]:
    """
    Intelligently route content based on automatic analysis of content type.
    Uses pattern matching and keyword analysis to determine the best agent.
    
    Args:
        content: Raw content to analyze and route
        context: Optional context to help with routing decision
        
    Returns:
        Dict containing:
        - agent_name: Name of the agent that processed the request
        - detected_type: Automatically detected content type
        - confidence: Confidence score of the routing decision (0.0 to 1.0)
        - response: Analysis from the most appropriate agent
        - status: Success or error status
        
    Example:
        >>> result = intelligent_routing("What is the latest stock price for TSLA?")
        >>> print(f"Routed to: {result['agent_name']}")
        >>> print(result['response'])
    """
    # Keywords for each content type with confidence weights
    content_lower = content.lower()
    context_lower = context.lower() if context else ""
    combined_text = f"{content_lower} {context_lower}"
    
    # Define keyword patterns with weights
    patterns = {
        "market": {
            "keywords": ["price", "stock", "quote", "ohlcv", "market data", "ticker", 
                        "volume", "technical", "chart", "trading"],
            "weight": 0.0
        },
        "earnings": {
            "keywords": ["earnings", "financial", "filings", "sec", "quarterly", 
                        "annual", "revenue", "profit", "balance sheet", "10-k", "10-q"],
            "weight": 0.0
        },
        "news": {
            "keywords": ["news", "article", "headlines", "sentiment", "recent", 
                        "announcement", "press release", "media"],
            "weight": 0.0
        },
        "memory": {
            "keywords": ["remember", "recall", "store", "forget", "save", 
                        "retrieve", "memory", "past"],
            "weight": 0.0
        },
        "evaluator": {
            "keywords": ["evaluate", "assess", "quality", "review", "feedback",
                        "score", "rating", "critique"],
            "weight": 0.0
        },
    }
    
    # Calculate weights based on keyword matches
    for category, data in patterns.items():
        for keyword in data["keywords"]:
            if keyword in combined_text:
                data["weight"] += 1.0
    
    # Regex patterns for specific formats (boost confidence)
    if re.search(r'\b[A-Z]{1,5}\b', content):  # Stock ticker pattern
        patterns["market"]["weight"] += 2.0
    
    if re.search(r'\b(10-[KQ]|8-K|earnings report|quarterly report)\b', content_lower):
        patterns["earnings"]["weight"] += 2.0
    
    # Determine the best match
    detected_type = max(patterns.items(), key=lambda x: x[1]["weight"])
    content_type = detected_type[0]
    confidence_score = detected_type[1]["weight"]
    
    # If no strong match found, default to investment research agent
    if confidence_score == 0.0:
        content_type = "investment"
        confidence_score = 0.5  # Medium confidence for default
    else:
        # Normalize confidence score (cap at 1.0)
        confidence_score = min(confidence_score / 5.0, 1.0)
    
    # Route to the determined agent
    result = route_content(content_type, content)
    
    # Add intelligent routing metadata
    result["detected_type"] = content_type
    result["confidence"] = round(confidence_score, 2)
    result["routing_method"] = "intelligent"
    
    return result


def multi_agent_routing(content: str, agent_types: list) -> Dict[str, Any]:
    """
    Route content to multiple agents and aggregate results.
    Useful for comprehensive analysis requiring multiple perspectives.
    
    Args:
        content: Content to analyze
        agent_types: List of agent types to route to (e.g., ["market", "news", "earnings"])
        
    Returns:
        Dict containing:
        - agents_used: List of agents that processed the request
        - responses: Dict mapping agent names to their responses
        - status: Overall status
        
    Example:
        >>> result = multi_agent_routing("AAPL", ["market", "news"])
        >>> for agent, response in result['responses'].items():
        >>>     print(f"{agent}: {response}")
    """
    results = {
        "agents_used": [],
        "responses": {},
        "status": "success"
    }
    
    for agent_type in agent_types:
        try:
            response = route_content(agent_type, content)
            if response["status"] == "success":
                results["agents_used"].append(response["agent_name"])
                results["responses"][response["agent_name"]] = response["response"]
            else:
                results["responses"][agent_type] = f"Error: {response['response']}"
                results["status"] = "partial_success"
        except Exception as e:
            results["responses"][agent_type] = f"Exception: {str(e)}"
            results["status"] = "partial_success"
    
    return results

### Evaluator-optimizer workflow

In [ ]:

READY_THRESHOLD = 0.85

_PRE_EVAL_MARK = "\n\n===DRAFT_BELOW==="

def _format_eval_matrix(feedback: dict) -> str:
    scores = feedback.get("scores", {}) or {}
    comp = float(scores.get("completeness", 0.0))
    acc  = float(scores.get("accuracy", 0.0))
    clr  = float(scores.get("clarity", 0.0))
    ovl  = float(scores.get("overall", (comp + acc + clr) / 3 if any([comp, acc, clr]) else 0.0))
    gaps = feedback.get("feedback", {}).get("gaps", []) or []

    rows = [
        "| Metric        | Score |",
        "|--------------|-------|",
        f"| Completeness | {comp:.2f} |",
        f"| Accuracy     | {acc:.2f} |",
        f"| Clarity      | {clr:.2f} |",
        f"| **Overall**  | **{ovl:.2f}** |",
        "",
        "#### Gaps",
        "- " + "\n- ".join(gaps) if gaps else "_None detected_",
    ]
    return "\n".join(rows), ovl

def show_pre_eval_metrics_executor(step_input) -> StepOutput:
    draft = _extract_text_from_ctx(step_input)
    feedback = _evaluate(draft)
    matrix_md, ovl = _format_eval_matrix(feedback)
    meta = f"\n\n===PRE_EVAL_OVERALL={ovl:.4f}==="
    content = "### Pre-Optimization Evaluation\n" + matrix_md + meta + _PRE_EVAL_MARK + "\n" + draft
    return StepOutput(content=content)


def _safe_json_loads(s: str):
    try:
        return json.loads(s)
    except Exception:
        return {"raw_response": s}

def _to_text(resp) -> str:
    if hasattr(resp, "content"):
        return resp.content
    if isinstance(resp, str):
        return resp
    if isinstance(resp, dict):
        return resp.get("content") or resp.get("delta") or str(resp)
    if isinstance(resp, types.GeneratorType) or hasattr(resp, "__iter__"):
        parts = []
        try:
            for chunk in resp:
                if hasattr(chunk, "content"):
                    parts.append(chunk.content)
                elif isinstance(chunk, str):
                    parts.append(chunk)
                elif isinstance(chunk, dict):
                    parts.append(chunk.get("content") or chunk.get("delta") or "")
                else:
                    parts.append(str(chunk))
        except Exception:
            pass
        return "".join(parts).strip()
    return str(resp)

def _extract_text_from_ctx(ctx) -> str:
    """
    ctx can be:
      - StepInput (has .previous_step_content / .content)
      - StepOutput (has .content)
      - list of StepOutput or mixed items (take last .content)
      - plain string
    """
    if hasattr(ctx, "previous_step_content") or hasattr(ctx, "content"):
        return (getattr(ctx, "previous_step_content", None)
                or getattr(ctx, "content", "")
                or "")
    if hasattr(ctx, "content"):
        return getattr(ctx, "content") or ""
    if isinstance(ctx, list) and ctx:
        for item in reversed(ctx):
            if hasattr(item, "content"):
                return item.content or ""
            if isinstance(item, str):
                return item
            if isinstance(item, dict) and "content" in item:
                return item["content"] or ""
        return ""
    return str(ctx or "")



def _compute_overall(scores: dict) -> float:
    keys = ["completeness", "accuracy", "clarity"]
    vals = [float(scores.get(k, 0.0)) for k in keys]
    return sum(vals) / max(len(vals), 1)

import re, json

def _json_from_messy(text: str):
    cleaned = "\n".join(line for line in (text or "").splitlines() if not line.strip().startswith("```")).strip()
    try:
        return json.loads(cleaned)
    except Exception:
        pass
    m = re.findall(r"\{[\s\S]*\}", cleaned)
    if m:
        for block in reversed(m):
            try:
                return json.loads(block)
            except Exception:
                repaired = block.replace(",]", "]").replace(",}", "}")
                try:
                    return json.loads(repaired)
                except Exception:
                    continue
    return {"raw_response": text}

def _evaluate(draft: str) -> dict:
    prompt = (
        "Evaluate the following financial analysis draft.\n"
        "Return ONLY the JSON object as per the schema.\n\n"
        "== DRAFT ==\n" + (draft or "").strip()
    )
    resp = evaluator_agent.run(prompt, stream=False)
    raw = _to_text(resp)
    data = _json_from_messy(raw)
    if isinstance(data, dict) and "scores" in data and "overall" not in data["scores"]:
        data["scores"]["overall"] = _compute_overall(data["scores"])
    print("==========   DEBUG  _evaluate  ================")
    print("[EVAL] draft_len:", len(draft), "raw_len:", len(raw), "keys:", list(data.keys()))
    return data


def _is_ready(feedback: dict, min_overall: float = READY_THRESHOLD) -> bool:
    try:
        scores = feedback.get("scores", {})
        overall = float(scores.get("overall", _compute_overall(scores)))
        gaps = feedback.get("feedback", {}).get("gaps", [])
        print("===========  EVAL RESULT  =========")
        print("scores: ", scores, "overall: ",overall, "gaps: ",gaps)
        return overall >= min_overall and not gaps
    except Exception:
        return False



OPT_GATE_THRESHOLD = 0.50
SKIP_OPT_MARK = "===SKIP_OPTIMIZATION==="

def gate_optimization_executor(step_input) -> StepOutput:
    mixed = _extract_text_from_ctx(step_input)

    if _PRE_EVAL_MARK in mixed:
        draft = mixed.split(_PRE_EVAL_MARK, 1)[1].lstrip()
    else:
        draft = mixed

    m = re.search(r"===PRE_EVAL_OVERALL=([0-9.]+)===", mixed)
    if m:
        overall = float(m.group(1))
    else:
        fb = _evaluate(draft)
        scores = fb.get("scores", {}) or {}
        overall = float(scores.get("overall", 0.0))

    # Gate: run optimization only if overall <= 0.50
    if overall <= OPT_GATE_THRESHOLD:
        # Allow loop to optimize: pass pure draft forward
        decision_banner = f"**Optimization Gate:** overall={overall:.2f} ≤ {OPT_GATE_THRESHOLD:.2f} → *OPTIMIZE*"
        return StepOutput(content=decision_banner + "\n\n" + draft)
    else:
        # Skip optimization: prepend a skip marker so the optimizer no-ops
        decision_banner = f"**Optimization Gate:** overall={overall:.2f} > {OPT_GATE_THRESHOLD:.2f} → *SKIP*"
        return StepOutput(content=decision_banner + "\n\n" + SKIP_OPT_MARK + "\n" + draft)


def evaluator_end_condition(ctx) -> bool:
    current_text = _extract_text_from_ctx(ctx)
    if current_text.lstrip().startswith(SKIP_OPT_MARK):
        return True
    print("[LOOP] end_condition: draft_len =", len(current_text))
    feedback = _evaluate(current_text)
    print("[LOOP] end_condition: parsed keys =", list(feedback.keys()))
    print("[LOOP] end_condition: scores =", feedback.get("scores"))
    return _is_ready(feedback, READY_THRESHOLD)


def optimizer_executor(step_input: StepInput) -> StepOutput:
    text = _extract_text_from_ctx(step_input).lstrip()

    if text.startswith(SKIP_OPT_MARK):
        draft = text.split("\n", 1)[1] if "\n" in text else ""
        feedback_after = _evaluate(draft)
        annotated = draft + "\n\n=== Evaluation Feedback ===\n" + json.dumps(feedback_after, indent=2) \
                    + "\n\n(Optimization skipped because overall > 0.50)"
        return StepOutput(content=annotated)

    parts = text.rsplit("=== Evaluation Feedback ===", 1)
    draft_only = parts[0].strip()

    prior_feedback = {}
    if len(parts) == 2:
        try:
            prior_feedback = json.loads(parts[1])
        except Exception:
            prior_feedback = {}

    optimizer_prompt = "\n".join([
        "You are improving the following financial research draft.",
        "Goals:",
        "- Address all gaps and priority fixes.",
        "- Improve completeness, accuracy, and clarity.",
        "- Keep structure tight; use headings and bullets sparingly.",
        "",
        "== DRAFT ==",
        draft_only,
        "",
        "== EVALUATION JSON ==",
        json.dumps(prior_feedback, indent=2),
        "",
        "== INSTRUCTIONS ==",
        "Rewrite the draft, directly producing the improved report with the fixes applied.",
        "Do not output a plan—output the revised analysis only.",
    ])

    improved_resp = investment_research_agent.run(optimizer_prompt, stream=False)
    improved_text = _to_text(improved_resp)
    feedback_after = _evaluate(improved_text)
    annotated = improved_text + "\n\n=== Evaluation Feedback ===\n" + json.dumps(feedback_after, indent=2)
    return StepOutput(content=annotated)


def _extract_text_from_ctx(ctx) -> str:
    if hasattr(ctx, "previous_step_content") or hasattr(ctx, "content"):
        base = (getattr(ctx, "previous_step_content", None)
                or getattr(ctx, "content", "")
                or "")
        return base

    if isinstance(ctx, list) and ctx:
        best = ""
        for item in ctx:
            if hasattr(item, "content") and isinstance(item.content, str):
                if len(item.content) > len(best):
                    best = item.content
            elif isinstance(item, str):
                if len(item) > len(best):
                    best = item
            elif isinstance(item, dict) and "content" in item and isinstance(item["content"], str):
                if len(item["content"]) > len(best):
                    best = item["content"]
        return best

    return str(ctx or "")


def finalize_print_executor(step_input) -> StepOutput:
    text = _extract_text_from_ctx(step_input)
    cleaned = text.split("\n\n=== Evaluation Feedback ===", 1)[0].strip()
    return StepOutput(content=cleaned)


eval_opt_loop = Loop(
    name="Evaluator–Optimizer Loop",
    steps=[Step(name="Optimize Draft", executor=optimizer_executor)],
    end_condition=evaluator_end_condition,
    max_iterations=1,
)


### Prompt Chaining and Evaluator-Optimizer Workflow

In [ ]:
# Block 11:
def parse_preprocessing_output(step_input: StepInput) -> dict:
    """
    Parses the JSON output from the preprocessing agent.

    Args:
        output (str): The JSON string output from the preprocessing agent.

    Returns:
        dict: Parsed dictionary containing ticker, action_item, and data_types.
    """
    try:
        output = step_input.get_step_content("Preprocess Input") or ""
        if not output.strip():
            return {}
        cleaned_output = "\n".join(
            line for line in output.splitlines() if not line.strip().startswith("```")
        )
        return loads(cleaned_output)
    except Exception as e:
        print(f"Error parsing preprocessing output: {e}")
        return {}


def multi_agent_router_executor(step_input: StepInput) -> StepOutput:
    """
    Executor function to route tasks to multiple agents based on input parameters.

    Args:
        step_input (StepInput): Input containing content and agent types.

    Returns:
        StepOutput: Output from the routed agents.
    """
    data = parse_preprocessing_output(step_input)
    ticker = data.get("ticker")
    action_item = data.get("action_item")
    agent_types = data.get("data_types", [])

    result = multi_agent_routing(f"Stock Ticker: {ticker}\n{action_item}", agent_types)
    output = ""
    for agent, response in result["responses"].items():
        output += f"{agent}: {response}\n\n"

    return StepOutput(content=output)


def stored_data_retriever_executor(step_input: StepInput) -> StepOutput:
    """
    Executor function to retrieve stored data from memory based on input parameters.

    Args:
        step_input (StepInput): Input containing content and agent types.

    Returns:
        StepOutput: Output from the memory agent.
    """
    data = parse_preprocessing_output(step_input)
    ticker = data.get("ticker")
    action_item = data.get("action_item")

    memory_prompt = "\n".join(
        [
            f"Retrieve any stored information related to {ticker} that might assist with the following request: {action_item}.",
            "This is what we already know, do not repeat it: ",
            step_input.previous_step_content,
            "If no relevant information is found, respond with 'No relevant information found.'",
        ]
    )
    memory_response = memory_agent.run(memory_prompt).content

    if "No relevant information found." not in memory_response:
        return StepOutput(
            content="\n".join(
                [
                    step_input.previous_step_content,
                    f"\n\nMemory Agent: {memory_response}",
                ]
            )
        )

    return StepOutput(content=step_input.previous_step_content)

def select_synthesis_executor(step_input) -> StepOutput:
    ctx = step_input.previous_step_content or step_input.content
    chosen = _extract_text_from_ctx(ctx)
    return StepOutput(content=chosen)

workflow = Workflow(
    name="Analysis workflow",
    steps=[
        Step(name="Preprocess Input", agent=preprocessing_agent),
        Step(
            name="Gather Data",
            executor=multi_agent_router_executor,
        ),
        Step(
            name="Retrieve Stored Data",
            executor=stored_data_retriever_executor,
        ),
        Parallel(
            Step(
                name="Synthesize Analysis",
                agent=investment_research_agent,
            ),
            Step(
                name="Store Data in Memory",
                agent=memory_agent,
            ),
        ),
        Step(name="Select Synthesis Output", executor=select_synthesis_executor),
        Step(name="Pre-Optimization Evaluation (Display)", executor=show_pre_eval_metrics_executor),
        Step(name="Gate Optimization (≤ 50% only)", executor=gate_optimization_executor),
        eval_opt_loop,  
        Step(name="Finalize & Print Optimized Report", executor=finalize_print_executor),
    ],
)



## Example Usage

In [104]:
workflow.print_response(
    (
        "Analyze the financial health and market position of Apple "
        "based on recent earnings reports, market data, and news articles."
    ),
    stream=True,
    stream_intermediate_steps=True,
)

┏━ Workflow Information ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Workflow: Analysis workflow                                                                                     ┃
┃                                                                                                                 ┃
┃ Steps: 9 steps                                                                                                  ┃
┃                                                                                                                 ┃
┃ Message: Analyze the financial health and market position of Apple based on recent earnings reports, market     ┃
┃ data, and news articles.                                                                                        ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.

Output()

┏━ Step 1: Preprocess Input (Completed) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃                                                                                                                 ┃
┃  {                                                                                                              ┃
┃    "ticker": "AAPL",                                                                                            ┃
┃    "action_item": "Analyze the financial health and market position of Apple",                                  ┃
┃    "data_types": [                                                                                              ┃
┃      "earnings",                                                                                                ┃
┃      "market",                                                                                                  ┃
┃      "news"                                                                                                     ┃
┃    ]                                                                                                            ┃
┃  }                                                                                                              ┃
┃                                                                                                                 ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

┏━ Step 2: Gather Data (Completed) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Earnings Agent: Earnings Analysis for Apple Inc. (AAPL)                                                         ┃
┃                                                                                                                 ┃
┃ Date of Analysis: 2025-10-07                                                                                    ┃
┃                                                                                                                 ┃
┃ Apple Inc. (AAPL) continues to demonstrate robust financial health driven by its powerful ecosystem, strong     ┃
┃ brand loyalty, and increasing penetration of its Services segment. While product sales can be cyclical, the     ┃
┃ company's strategic focus on services and ongoing capital return programs underpin its market position.         ┃
┃                                                                                                                 ┃
┃ ─────────────────────────────────────────────────────────────────────────────────────────────────────────────── ┃
┃                                      1. Revenue and Earnings Growth Trends                                      ┃
┃                                                                                                                 ┃
┃ Apple's revenue growth has shown resilience, often outperforming broader market trends, though it can be        ┃
┃ influenced by iPhone product cycles and macroeconomic conditions.                                               ┃
┃                                                                                                                 ┃
┃  • Revenue Growth: For the fiscal year ending September 2024 (FY2024), Apple reported total net sales of        ┃
┃    approximately $395 billion, representing a 4% year-over-year increase. This reflects a stabilization and     ┃
┃    modest recovery after some periods of softer growth, particularly in iPhone sales in certain regions. The    ┃
┃    Services segment has been a consistent growth driver.                                                        ┃
┃  • Earnings Growth: Diluted earnings per share (EPS) for FY2024 reached approximately $6.60, up 7% from the     ┃
┃    previous year. This stronger EPS growth relative to revenue is often attributed to effective cost            ┃
┃    management, an increasing mix of higher-margin services, and aggressive share repurchases which reduce the   ┃
┃    share count.                                                                                                 ┃
┃  • Gross Margin: Gross margins have remained strong and stable, typically hovering around 44-45%, indicating    ┃
┃    efficient cost control and pricing power for both products and services. Services consistently command       ┃
┃    higher gross margins (typically 70%+) compared to products (around 35-40%).                                  ┃
┃                                                                                                                 ┃
┃ Insight: Apple demonstrates consistent profitability and a solid top-line, though growth rates can fluctuate.   ┃
┃ The Services segment is key to stable, predictable revenue streams and margin expansion, offsetting some        ┃
┃ volatility in product sales.                                                                                    ┃
┃                                                                                                                 ┃
┃ ─────────────────────────────────────────────────────────────────────────────────────────────────────────────── ┃
┃                                             2. Key Financial Ratios                                             ┃
┃                                                       

...

===========  EVAL RESULT  =========

scores:  {'completeness': 0.9, 'accuracy': 0.8, 'clarity': 0.95, 'overall': 0.883} overall:  0.883 gaps:  
["Critical Financial Data Verification: The accuracy of specific FY2024 and Q3 FY2025 financial figures (revenue, 
EPS, segment breakdowns, ratios, cash flow) presented as factual as of 2025-10-07 cannot be verified without 
external access to Apple's official, reported filings from those periods.", "Technical Indicator Specificity: The 
analysis of technical indicators (Moving Averages, RSI, MACD, Bollinger Bands) relies on somewhat speculative 
language ('likely remains,' 'might be contracting'), indicating a lack of explicit real-time data for the analysis 
date.", 'Valuation Methodology Detail: While a premium P/E range (30x-32x) is provided, the draft does not 
explicitly detail the underlying valuation models (e.g., DCF, relative valuation against peers) used to support 
this claim or help identify precise entry/exit points.', "Peer Comparison Integration: A brief, explicit comparison
of Apple's key financial metrics or market positioning against direct competitors could further strengthen the 
market positioning and risk assessment sections."]

┏━ Loop Evaluator–Optimizer Loop (Completed) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Loop Summary:                                                                                                   ┃
┃                                                                                                                 ┃
┃  • Total iterations: 1/1                                                                                        ┃
┃  • Total steps executed: 1                                                                                      ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

┏━ Step 9: Finalize & Print Optimized Report (Completed) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃                                                                                                                 ┃
┃                                 Investment Research Workflow: Apple Inc. (AAPL)                                 ┃
┃                                                                                                                 ┃
┃ Date of Analysis: 2025-10-07                                                                                    ┃
┃                                                                                                                 ┃
┃ Objective: To conduct a comprehensive financial and market analysis of Apple Inc. (AAPL) to derive actionable   ┃
┃ investment insights and recommendations.                                                                        ┃
┃                                                                                                                 ┃
┃ ─────────────────────────────────────────────────────────────────────────────────────────────────────────────── ┃
┃                                                Executive Summary                                                ┃
┃                                                                                                                 ┃
┃ Apple (AAPL) continues to demonstrate formidable financial strength, driven by its robust ecosystem, dominant   ┃
┃ brand, and rapidly expanding Services segment. FY2024 saw stable revenue growth (4%) and strong EPS growth      ┃
┃ (7%), supported by efficient cost management and aggressive share repurchases. While its valuation remains      ┃
┃ premium (P/E ~30-32x), justified by immense free cash flow generation and consistent capital returns, the       ┃
┃ company faces increasing regulatory scrutiny, supply chain dependencies, and intense competition. Strategic     ┃
┃ investments in AI and spatial computing (Vision Pro) offer significant long-term growth potential,              ┃
┃ counteracting some of the cyclicality in its core iPhone business. Investors should acknowledge the premium     ┃
┃ valuation and monitor regulatory risks closely, considering AAPL as a core holding for long-term growth with a  ┃
┃ focus on its ecosystem and services expansion, particularly amidst recent volatility stemming from analyst      ┃
┃ downgrades and leadership uncertainty.                                                                          ┃
┃                                                                                                                 ┃
┃ ─────────────────────────────────────────────────────────────────────────────────────────────────────────────── ┃
┃                                       Research Workflow Steps & Analysis                                        ┃
┃                                                                                                                 ┃
┃                                             1. Market Data Analysis                                             ┃
┃                                                                                                                 ┃
┃ Data Sources Consulted: Market Data Agent, News Agent, Memory Agent                                             ┃
┃                                                                                                                 ┃
┃ Workflow:                                                                                                       ┃
┃                                                                                                                 ┃
┃  • 1.1. Current Price & Immediate Context: AAPL is currently trading at $256.69 USD. Recent news (2025-09-22)   ┃
┃    indicated the stock was "Nearing Record" highs and 

Completed in 233.4s